In [1]:
import numpy as np
import pandas as pd
import os
import scipy.io as sio
import scipy.misc
import pickle

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 10.0) # set default size of plots
#plt.rcParams['image.interpolation'] = 'nearest'
#plt.rcParams['image.cmap'] = 'gray'

In [2]:
inpath = '/home/tao/deep_learning/text-to-image-master/Data/birds'

In [4]:
# <image_id> <image_name>
df_filenames = pd.read_csv(os.path.join(inpath, 'images.txt'), delim_whitespace=True,header=None)
df_filenames.head()

,0,1
0,1,001.Black_footed_Albatross/Black_Footed_Albatr...
1,2,001.Black_footed_Albatross/Black_Footed_Albatr...
2,3,001.Black_footed_Albatross/Black_Footed_Albatr...
3,4,001.Black_footed_Albatross/Black_Footed_Albatr...
4,5,001.Black_footed_Albatross/Black_Footed_Albatr...


In [8]:
labels = df_filenames[1].tolist()
print labels[:5]

['001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg', '001.Black_footed_Albatross/Black_Footed_Albatross_0009_34.jpg', '001.Black_footed_Albatross/Black_Footed_Albatross_0002_55.jpg', '001.Black_footed_Albatross/Black_Footed_Albatross_0074_59.jpg', '001.Black_footed_Albatross/Black_Footed_Albatross_0014_89.jpg']


In [4]:
#<image_id> <x> <y> <width> <height>
df_bounding_boxes = pd.read_csv(os.path.join(inpath, 'bounding_boxes.txt'), delim_whitespace=True,header=None).astype(int)
df_bounding_boxes.head()

,0,1,2,3,4
0,1,60,27,325,304
1,2,139,30,153,264
2,3,14,112,388,186
3,4,112,90,255,242
4,5,70,50,134,303


In [20]:
arr_attributes = sio.loadmat(os.path.join(inpath, 'attributesVec.mat'))['attributesVec']
print type(arr_attributes), arr_attributes.shape

attributesShortNames = pd.read_csv(os.path.join(inpath, 'attributesShortNames.txt'), delim_whitespace=True,header=None)
attributesShortNames.head()

<type 'numpy.ndarray'> (11788, 312)


,0,1,2
0,1,bill,curved
1,2,bill,dagger
2,3,bill,hooked
3,4,bill,needle
4,5,bill,hooked_seabird


In [29]:
valid_attributesShortNames = pd.DataFrame()
print valid_attributesShortNames.head()

valid_names = ['primary_color', 'shape', 'size']
valid_dim = []
for i in range(len(attributesShortNames)):
    row = attributesShortNames.iloc[i]
    if row[1] in valid_names:
        valid_dim.append(row[0])
        valid_attributesShortNames = valid_attributesShortNames.append(row, ignore_index=True)
        #print row.tolist()
print len(valid_dim), valid_dim
print len(valid_attributesShortNames), valid_attributesShortNames.head()   

valid_arr_attributes = arr_attributes[:, valid_dim]
print valid_arr_attributes.shape, arr_attributes[0,218:264], valid_arr_attributes[0,:]

(11788, 34) [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0] [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]


In [36]:
numImgs = valid_arr_attributes.shape[0]
attributes_labels = []
for i in xrange(0, numImgs):
    #print i
    s = ''
    idx = np.where(valid_arr_attributes[i,:]==1)[0]
    #print type(idx), len(idx), idx
    if len(idx)>0:
        df_hasAttr = valid_attributesShortNames.iloc[idx].sort(columns=[1])
        #print type(df_hasAttr.index), len(df_hasAttr.index), df_hasAttr.index
        part = ''
        for j in df_hasAttr.index:
            row = valid_attributesShortNames.iloc[j]
            if part!=row[1]:
                #print s
                part=row[1]
                s += '\n' + part + ': ' + row[2]
            else:
                s += ';' + row[2]
    #print s
    attributes_labels.append(s)
print len(attributes_labels), attributes_labels[:5]

11788 ['\nprimary_color: red\nshape: chicken-like-marsh', '\nprimary_color: rufous\nshape: swallow-like\nsize: large', '\nprimary_color: blue\nshape: owl-like\nsize: small', '\nprimary_color: blue\nshape: owl-like;perching-like\nsize: very_large', '\nprimary_color: blue;orange\nshape: owl-like']


In [38]:
bcrop = 0
bsave = 0
numImgs = valid_arr_attributes.shape[0]
IMSIZE = 64

height = IMSIZE
width = IMSIZE
depth = 3
embedding_num = 1
images = []
embeddings = valid_arr_attributes
labels = df_filenames[1].tolist()
for i in xrange(0, numImgs): 
    filepath = os.path.join(inpath, 'images_cropped/'+df_filenames.iloc[i][1])
    print filepath
    img = scipy.misc.imread(filepath)
    #print img.shape
    
    
    
    #<x> <y> <width> <height>
    if bcrop:
        bbox = df_bounding_boxes.iloc[i][1:].tolist()
    else:
        bbox = [0, 0, img.shape[1], img.shape[0]]
    c_y = int((bbox[1]+bbox[3]/2))
    c_x = int((bbox[0]+bbox[2]/2))   
    imR = int(np.min(np.asarray([c_y, c_x, img.shape[0]-c_y, img.shape[1]-c_x, np.maximum(bbox[2]/2, bbox[3]/2)])))
    #print bbox
    #print c_y, c_x, imR
    resized_img = scipy.misc.imresize(img[c_y-imR:c_y+imR, c_x-imR:c_x+imR], [IMSIZE, IMSIZE])
    #print resized_img.shape #, resized_img
    if len(resized_img.shape)==2:
        resized_img = resized_img.reshape(resized_img.shape[0], resized_img.shape[1], 1)
        resized_img = np.concatenate([resized_img, resized_img, resized_img], axis=2)
    if len(resized_img.shape)==4:
        resized_img = resized_img[:, :, 0:3]
    
    #resized_img = resized_img.astype('float64')
    #resized_img += 1.
    #resized_img *= (255. / 2.)
    #resized_img = resized_img.astype('uint8')
        
    if bsave:
        print('%d\t%s' % (i, labels[i]))
        images.append(resized_img)
    else:
        fig = plt.figure()
        plt.subplot(1,2,1)
        plt.imshow(img)
        plt.gca().axis('off')
        plt.subplot(1,2,2)
        plt.imshow(resized_img)
        plt.gca().axis('off')
        #plt.show()
        break
        
if bsave:
    with open(os.path.join(inpath, 'birds64_more.pickle'), 'wb') as f_out:
        pickle.dump([height, width, depth, embedding_num, images,
                    embeddings, labels, attributes_labels], f_out)

0	001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg
1	001.Black_footed_Albatross/Black_Footed_Albatross_0009_34.jpg
2	001.Black_footed_Albatross/Black_Footed_Albatross_0002_55.jpg
3	001.Black_footed_Albatross/Black_Footed_Albatross_0074_59.jpg
4	001.Black_footed_Albatross/Black_Footed_Albatross_0014_89.jpg
5	001.Black_footed_Albatross/Black_Footed_Albatross_0085_92.jpg
6	001.Black_footed_Albatross/Black_Footed_Albatross_0031_100.jpg
7	001.Black_footed_Albatross/Black_Footed_Albatross_0051_796103.jpg
8	001.Black_footed_Albatross/Black_Footed_Albatross_0010_796097.jpg
9	001.Black_footed_Albatross/Black_Footed_Albatross_0025_796057.jpg
10	001.Black_footed_Albatross/Black_Footed_Albatross_0023_796059.jpg
11	001.Black_footed_Albatross/Black_Footed_Albatross_0086_796062.jpg
12	001.Black_footed_Albatross/Black_Footed_Albatross_0049_796063.jpg
13	001.Black_footed_Albatross/Black_Footed_Albatross_0006_796065.jpg
14	001.Black_footed_Albatross/Black_Footed_Albatross_0040_796066.jpg
15

(64, 64, 3)
